In [1]:
import pandas as pd

In [2]:
usuarios_datos_path = "./rawdata/usuarios_datos_personales.txt"
with open(usuarios_datos_path, "r", encoding="latin1") as file:
    usuarios_datos_content = file.readlines()

# Extracting valid user records by enforcing strict 8-field grouping (5 known + 3 additional fields)
cleaned_records = []
current_record = []

for line in usuarios_datos_content:
    line = line.strip()

    if len(current_record) == 0 and line.isdigit():  # First field: id_usuario
        current_record.append(line)
    elif len(current_record) == 1 and not line.isdigit():  # Second field: nombre_usuario (non-numeric)
        current_record.append(line)
    elif len(current_record) == 2 and line.isdigit():  # Third field: edad (numeric)
        current_record.append(line)
    elif len(current_record) == 3 and not line.isdigit():  # Fourth field: sexo (non-numeric, usually 'F' or 'M')
        current_record.append(line)
    elif len(current_record) == 4 and line.isdigit():  # Fifth field: ocupacion_id (numeric)
        current_record.append(line)
    elif len(current_record) in [5, 6, 7] and line.isdigit():  # Next three fields: Hijos, Edad hijo menor, Edad hijo mayor
        current_record.append(line)

    if len(current_record) == 8:  # Once we have all required fields, save the record
        cleaned_records.append(current_record)
        current_record = []

# Creating a DataFrame with correctly structured user records
usuarios_datos = pd.DataFrame(cleaned_records, columns=[
    "id_usuario", "nombre_usuario", "edad", "sexo", "id_ocupacion",
    "hijos", "edad_hijo_menor", "edad_hijo_mayor"
])

# Convert data types
usuarios_datos["id_usuario"] = usuarios_datos["id_usuario"].astype(int)
usuarios_datos["edad"] = usuarios_datos["edad"].astype(int)
usuarios_datos["id_ocupacion"] = usuarios_datos["id_ocupacion"].astype(int)
usuarios_datos["hijos"] = usuarios_datos["hijos"].astype(int)
usuarios_datos["edad_hijo_menor"] = usuarios_datos["edad_hijo_menor"].astype(int)
usuarios_datos["edad_hijo_mayor"] = usuarios_datos["edad_hijo_mayor"].astype(int)

In [3]:
ocupaciones_path = "./rawdata/ocupaciones.txt"
with open(ocupaciones_path, "r", encoding="latin1") as file:
    ocupaciones_content = file.readlines()

# Restructuring the occupations data (every other line)
ocupacion_ids = ocupaciones_content[0::2]
ocupacion_names = ocupaciones_content[1::2]

# Creating occupations DataFrame
ocupaciones = pd.DataFrame({
    "id_ocupacion": [int(x.strip()) for x in ocupacion_ids],
    "ocupacion": [x.strip() for x in ocupacion_names]
})

# Merging user data with occupation names
usuarios_datos = usuarios_datos.merge(ocupaciones, on="id_ocupacion", how="left")
usuarios_datos.head(5)

,id_usuario,nombre_usuario,edad,sexo,id_ocupacion,hijos,edad_hijo_menor,edad_hijo_mayor,ocupacion
0,123,Usuario_123,37,F,3,1,3,8,Técnicos y profesionales científicos e intelec...
1,124,Usuario_124,42,F,8,1,3,8,Artesanos y trabajadores cualificados de indus...
2,125,Usuario_125,35,F,10,0,0,0,Trabajadores no cualificados
3,126,Usuario_126,50,F,4,0,0,0,Técnicos y profesionales de apoyo
4,127,Usuario_127,66,F,11,1,35,44,Inactivo o desocupado


In [5]:
usuarios_prefs_path = "./rawdata/usuarios_preferencias.txt"
with open(usuarios_prefs_path, "r", encoding="latin1") as file:
    usuarios_prefs_content = file.readlines()

# Restructuring user preferences into records of 3 values each
usuarios_prefs_records = [usuarios_prefs_content[i:i+3] for i in range(0, len(usuarios_prefs_content), 3)]

# Creating a DataFrame with correct columns
usuarios_prefs = pd.DataFrame(usuarios_prefs_records, columns=["id_usuario", "id_categoria", "calificacion"])

# Cleaning up newline characters and ensuring correct data types
usuarios_prefs = usuarios_prefs.applymap(lambda x: x.strip() if isinstance(x, str) else x)
usuarios_prefs["id_usuario"] = usuarios_prefs["id_usuario"].astype(int)
usuarios_prefs["id_categoria"] = usuarios_prefs["id_categoria"].astype(int)
usuarios_prefs["calificacion"] = usuarios_prefs["calificacion"].astype(int) * 10  # Convert scale 1-10 to 1-100

# Load preferences categories
categorias_path = "./rawdata/preferencias.txt"
with open(categorias_path, "r", encoding="latin1") as file:
    categorias_content = file.readlines()

# Correcting the parsing of the preferences file
categoria_ids = []
categoria_names = []

for i in range(0, len(categorias_content), 3):  # Every 3 lines, extract relevant data
    if i + 1 < len(categorias_content):  # Ensure we don't go out of bounds
        categoria_ids.append(int(categorias_content[i].strip()))
        categoria_names.append(categorias_content[i + 1].strip())

# Creating corrected preferences DataFrame
categorias = pd.DataFrame({
    "id_categoria": categoria_ids,
    "categoria": categoria_names
})

# Merging preferences with descriptions
usuarios_prefs_final = usuarios_prefs.merge(categorias, on="id_categoria", how="left")
usuarios_prefs_final

C:\Users\valel\AppData\Local\Temp\ipykernel_12128\416502.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  usuarios_prefs = usuarios_prefs.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,id_usuario,id_categoria,calificacion,categoria
0,123,16,80,Prehistórico
1,123,17,90,Árabe
2,123,18,90,Romano
3,123,19,80,Medieval
4,123,20,100,Gótico
...,...,...,...,...
5465,180,105,90,Esculturas
5466,180,106,90,Fuentes
5467,180,107,90,Otros monumentos
5468,180,108,60,Centros de salud y spa


In [ ]:
usuarios_datos.to_csv("./data/user_info.csv", index=False, encoding="utf-8")
usuarios_prefs_final.to_csv("./data/user_preference.csv", index=False, encoding="utf-8")